#  Load Hitters

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

import os
from dotenv import load_dotenv
import psycopg2

from sim import Player

load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASS"),
    port=os.getenv("DB_PORT"),
    sslmode=os.getenv("DB_SSLMODE", "require"),
    sslrootcert=os.getenv("DB_SSLROOTCERT")
)

# Enable SSL but skip server certificate verification (rejectUnauthorized = 0 equivalent)
# conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?sslmode=require"
# engine = create_engine(conn_str)

sql1 = """
SELECT *
FROM dev_bwoodrum.ws_2025_hit
"""

# with engine.connect() as conn:
#     hit_df = pd.read_sql_query(sql1, conn)

hit_df = pd.read_sql_query(sql1, conn)

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
output_dir = os.path.join(project_root, "ws_2025_monte_carlo/data")
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "hitter_probs.csv")
# hit_df.to_csv(output_path, index=False)
# print(f"✅ Saved hitter_df to {output_path}")

# --- Convert hitter DataFrame to Player objects ---
hitters = []
for _, row in hit_df.iterrows():
    hitters.append(
        Player(
            name=row["full_name"],         # full name
            hand=row.get("bats", "R"),     # handedness
            probs=row.to_dict(),           # use full row as probability lookup dict
            contact=row.get("contact", 0.0),
            power=row.get("power", 0.0),
            speed=row.get("speed", 0.0),
        )
    )

print(f"✅ Loaded {len(hitters)} hitters into Player objects.")
print(hitters[:5])


# QA Hitters

In [ ]:
# Inspect the result
hit_df.head()


In [8]:
hit_df.columns

Index(['season', 'bpid', 'mlbam_id', 'full_name', 'pit_hand', 'bats', 'team',
       'pa', 'out_rate_pred', 'so_rate_pred', 'bb_rate_pred', 'hbp_rate_pred',
       'roe_rate_pred', 'single_rate_pred', 'double_rate_pred',
       'triple_rate_pred', 'hr_rate_pred', 'drc_plus'],
      dtype='object')

# Load Pitchers

In [ ]:
import os
import pandas as pd
from sim import Pitcher 

# --- Load from database ---
sql2 = """
SELECT *
FROM dev_bwoodrum.ws_2025_pit
"""
pit_df = pd.read_sql_query(sql2, conn)

# --- Save to CSV (optional) ---
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
output_dir = os.path.join(project_root, "ws_2025_monte_carlo/data")
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "pitcher_probs.csv")
pit_df.to_csv(output_path, index=False)
print(f"✅ Saved pitcher_df to {output_path}")

# --- Build Pitcher objects ---
pitchers = []
for _, row in pit_df.iterrows():
    pitchers.append(
        Pitcher(
            name=row["full_name"],
            hand=row["pit_hand"],
            dra_minus=row.get("dra_minus", 100),
            dra_minus_L=row.get("dra_minus_L", row.get("dra_minus", 100)),
            dra_minus_R=row.get("dra_minus_R", row.get("dra_minus", 100)),
        )
    )

print(f"✅ Loaded {len(pitchers)} pitchers")
print(pitchers[:5])  # preview first few objects


# QA Pitchers

In [7]:
pit_df.columns

Index(['season', 'bpid', 'mlbam_id', 'full_name', 'pit_hand', 'bats', 'team',
       'bf', 'out_rate_pred', 'so_rate_pred', 'bb_rate_pred', 'hbp_rate_pred',
       'roe_rate_pred', 'single_rate_pred', 'double_rate_pred',
       'triple_rate_pred', 'hr_rate_pred', 'dra_minus'],
      dtype='object')

In [ ]:
from sim.load_data import load_hitters, load_pitchers

hitters = load_hitters("data/hitter_probs.csv")
pitchers = load_pitchers("data/pitcher_probs.csv")

print(hitters[:5])
print(pitchers[:5])